In [8]:
def dice_loss(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    dice = (2. * intersection + K.epsilon()) / (union + K.epsilon())
    return 1 - dice
def f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = TP / (Positives+K.epsilon())
        return recall
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = TP / (Pred_Positives+K.epsilon())
        return precision
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import networkx as nx
from skimage.morphology import skeletonize
from PIDController import PIDController
from OT2Eenv import OT2Env  # Updated environment class


def crop_petri_dish(image, patch_size):
    """
    Detect and crop the Petri dish from the image.

    Parameters:
    - image: Input image (numpy array).
    - patch_size: Tuple (height, width) to pad the cropped Petri dish.

    Returns:
    - Cropped image focused on the Petri dish.
    - Bounding box of the Petri dish.
    - Success flag.
    """
    # Threshold the image to create a binary mask
    _, binary = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY)

    # Find contours in the binary mask
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Detect the largest contour as the Petri dish
    largest_contour = max(contours, key=cv2.contourArea, default=None)
    if largest_contour is None:
        print("Error: No Petri dish detected.")
        return None, None, False

    # Get the bounding box of the Petri dish
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Crop the image based on the bounding box
    cropped_image = image[y:y + h, x:x + w]

    # Pad the cropped image to ensure it matches the patch size
    padded_image = pad_image(cropped_image, patch_size)

    return padded_image, (x, y, w, h), True


def pad_image(image, patch_size):
    """
    Pad the cropped image to match the required patch size.

    Parameters:
    - image: Input cropped image (numpy array).
    - patch_size: Tuple (height, width) for padding.

    Returns:
    - Padded image.
    """
    height, width = image.shape[:2]
    pad_height = (patch_size[0] - height % patch_size[0]) % patch_size[0]
    pad_width = (patch_size[1] - width % patch_size[1]) % patch_size[1]
    return cv2.copyMakeBorder(image, 0, pad_height, 0, pad_width, cv2.BORDER_CONSTANT, value=0)


def split_image(image, num_parts):
    """
    Split the image into equal vertical parts.

    Parameters:
    - image: Input image (numpy array).
    - num_parts: Number of parts to split the image into.

    Returns:
    - List of image splits.
    """
    height, width = image.shape[:2]
    part_width = width // num_parts
    return [image[:, i * part_width:(i + 1) * part_width] for i in range(num_parts)]


def merge_images(splits, original_shape):
    """
    Merge image splits back into the original shape.

    Parameters:
    - splits: List of image splits.
    - original_shape: Tuple (height, width) of the original image.

    Returns:
    - Merged image.
    """
    merged_image = np.zeros(original_shape, dtype=np.uint8)
    part_width = original_shape[1] // len(splits)
    for i, split in enumerate(splits):
        merged_image[:, i * part_width:(i + 1) * part_width] = split
    return merged_image


def generate_mask(image, model, patch_size):
    """
    Generate a binary mask from an input image using the trained model.

    Parameters:
    - image: Input image (numpy array).
    - model: Trained Keras model.
    - patch_size: Tuple (height, width) for patching.

    Returns:
    - mask: Predicted binary mask (numpy array).
    """
    height, width = image.shape[:2]
    # Pad the image
    pad_height = (patch_size[0] - height % patch_size[0]) % patch_size[0]
    pad_width = (patch_size[1] - width % patch_size[1]) % patch_size[1]
    padded_image = cv2.copyMakeBorder(image, 0, pad_height, 0, pad_width, cv2.BORDER_CONSTANT, value=0)

    # Patch the image
    patches = []
    for y in range(0, padded_image.shape[0], patch_size[0]):
        for x in range(0, padded_image.shape[1], patch_size[1]):
            patch = padded_image[y:y + patch_size[0], x:x + patch_size[1]]
            patches.append(patch / 255.0)  # Normalize

    patches = np.array(patches)[..., np.newaxis]  # Add channel dimension

    # Predict patches
    predicted_patches = model.predict(patches)
    predicted_patches = (predicted_patches > 0.5).astype(np.uint8) * 255

    # Reconstruct the mask
    reconstructed_mask = np.zeros_like(padded_image, dtype=np.uint8)
    idx = 0
    for y in range(0, padded_image.shape[0], patch_size[0]):
        for x in range(0, padded_image.shape[1], patch_size[1]):
            reconstructed_mask[y:y + patch_size[0], x:x + patch_size[1]] = predicted_patches[idx].squeeze()
            idx += 1

    # Crop back to the original size
    return reconstructed_mask[:height, :width]


def detect_root_tip_with_skeletonization(mask, kernel_size=10, closing_iterations=3, min_area=400):
    """
    Detect the root tip using the improved skeletonization process.

    Parameters:
    - mask: Binary mask (numpy array).
    - kernel_size: Kernel size for morphological operations.
    - closing_iterations: Number of iterations for morphological closing.
    - min_area: Minimum area for connected components.

    Returns:
    - root_tip: Tuple (y_pixel, x_pixel) of the root tip.
    """
    # Improve connectivity
    improved_mask = improve_connectivity(mask, kernel_size, closing_iterations)

    # Extract large connected components
    large_components = extract_large_components(improved_mask, min_area=min_area)

    # Find the longest path in the largest component
    longest_path = None
    max_length = 0

    for label, component_mask, area, stats in large_components:
        path, length, skeleton = find_longest_path_in_component(component_mask)
        if length > max_length:
            max_length = length
            longest_path = path

    if not longest_path:
        raise ValueError("No valid root tip detected in the skeletonized mask.")

    # The root tip is the endpoint of the longest path (lowest pixel)
    root_tip = longest_path[-1]
    return root_tip

def improve_connectivity(mask, kernel_size=5, closing_iterations=3):
    """
    Improves connectivity in the binary mask by applying morphological closing.
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    improved_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=closing_iterations)
    return improved_mask


def skeleton_to_graph(skeleton):
    """Converts a skeletonized image to a graph using networkx."""
    G = nx.Graph()
    skeleton_pixels = np.argwhere(skeleton > 0)
    for y, x in skeleton_pixels:
        G.add_node((y, x))
        for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            neighbor = (y + dy, x + dx)
            if (neighbor[0] >= 0 and neighbor[1] >= 0 and neighbor in G.nodes):
                G.add_edge((y, x), neighbor)
    return G


def find_longest_path_in_component(component_mask):
    """
    Finds the longest path in a single connected component using its skeleton.
    """
    skeleton = skeletonize(component_mask // 255)
    G = skeleton_to_graph(skeleton)
    topmost_pixel = tuple(np.argwhere(skeleton > 0).min(axis=0))

    if topmost_pixel not in G.nodes:
        skeleton_pixels = np.array(list(G.nodes))
        distances = np.sum(np.abs(skeleton_pixels - np.array(topmost_pixel)), axis=1)
        closest_node = tuple(skeleton_pixels[np.argmin(distances)])
        topmost_pixel = closest_node

    lengths = nx.single_source_dijkstra_path_length(G, source=topmost_pixel)
    bottommost_pixel = max(lengths, key=lengths.get)
    longest_path = nx.shortest_path(G, source=topmost_pixel, target=bottommost_pixel)
    return longest_path, lengths[bottommost_pixel], skeleton


def extract_large_components(mask, min_area=500):
    """
    Extracts connected components larger than a specified area.
    """
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    large_components = []
    for label in range(1, num_labels):  # Skip background
        area = stats[label, cv2.CC_STAT_AREA]
        if area >= min_area:
            component_mask = np.zeros_like(mask, dtype=np.uint8)
            component_mask[labels == label] = 255
            large_components.append((label, component_mask, area, stats[label]))
    return large_components

def convert_pixel_to_mm(root_tip_pixel, image_height, plate_height_mm):
    """
    Convert pixel coordinates to millimeters.

    Parameters:
    - root_tip_pixel: Root tip coordinates in pixels (y_pixel, x_pixel).
    - image_height: Height of the original image in pixels.
    - plate_height_mm: Real-world height of the plate in millimeters.

    Returns:
    - root_tip_mm: Root tip coordinates in millimeters (x_mm, y_mm, z_mm).
    """
    scale = plate_height_mm / image_height  # mm per pixel
    y_mm = root_tip_pixel[0] * scale
    x_mm = root_tip_pixel[1] * scale
    return (x_mm, y_mm, 0)  


def convert_to_robot_coordinates(root_tip_mm, plate_position_robot):
    """
    Convert root tip positions in mm (relative to the plate) to robot space.

    Parameters:
    - root_tip_mm: Root tip position in mm (x_mm, y_mm, z_mm).
    - plate_position_robot: Position of the top-left corner of the plate in robot space [x, y, z].

    Returns:
    - root_tip_robot: Root tip position in robot space (x_robot, y_robot, z_robot).
    """
    # Convert from mm to meters
    root_tip_m = [
        root_tip_mm[0] / 1000,
        root_tip_mm[1] / 1000,
        root_tip_mm[2] / 1000,
    ]

    # Add the plate position to get the robot space coordinates
    return [
        root_tip_m[0] + plate_position_robot[0],
        root_tip_m[1] + plate_position_robot[1],
        root_tip_m[2] + plate_position_robot[2],
    ]


def inoculate_with_pid(env, root_tips_robot):
    """
    Perform root tip inoculation using the PID controller, ensuring X and Y accuracy.

    Parameters:
    - env: The simulation environment.
    - root_tips_robot: List of root tip coordinates in robot space [(x, y, z), ...].
    """
    # Initialize PID controllers for X, Y, Z axes
    pid_x = PIDController(kp=10, ki=5, kd=0.01)
    pid_y = PIDController(kp=10, ki=5, kd=0.01)
    pid_z = PIDController(kp=10, ki=5, kd=0.01)

    for idx, root_tip in enumerate(root_tips_robot):
        print(f"Starting inoculation for root tip {idx + 1} at {root_tip}")
        terminated = False
        truncated = False
        env.goal_position = np.array(root_tip)  # Set the goal position for the environment

        while not (terminated or truncated):
            current_position = np.array(env.get_current_position())  # Robot's current position
            print(f"Current position: {current_position}, Goal position: {env.goal_position}")

            # Compute errors
            error_x = env.goal_position[0] - current_position[0]
            error_y = env.goal_position[1] - current_position[1]
            error_z = env.goal_position[2] - current_position[2]

            # PID outputs
            control_x = pid_x.compute(error_x)
            control_y = pid_y.compute(error_y)
            control_z = pid_z.compute(error_z)

            # Take action
            action = np.array([control_x, control_y, control_z], dtype=np.float32)
            obs, reward, terminated, truncated, info = env.step(action)

            # Render and display progress
            env.render()

            # Log the accuracy for X and Y
            xy_error = np.linalg.norm(env.goal_position[:2] - current_position[:2])
            print(f"XY Control: Current XY: {current_position[:2]}, Target XY: {env.goal_position[:2]}, Error XY: {xy_error}")

            # Check if the robot is within the acceptable X and Y error threshold
            if xy_error < 0.001:  # Example threshold
                print(f"Inoculating at position {current_position[:2]} (XY Accuracy Met)")
                print("Simulating inoculum drop...")  # Print message as a placeholder for the drop action
                break

        print(f"Finished inoculation for root tip {idx + 1}\n")




# Main Workflow
if __name__ == "__main__":
    # Initialize environment and parameters
    env = OT2Env(render=True)
    
    # Parameters
    model_path = r"C:\Users\Edopi\Desktop\2024-25b-fai2-adsai-EdoardoPierezza231412\datalab_tasks\Task8\Edoardo_231412_undet_model256px_base.h5"
    patch_size = (256, 256)
    plate_position_robot = [0.10775, 0.088 - 0.026, 0.057]  # Adjusted plate position
    image_height = 2816 
    plate_height_mm = 150  

    # Capture the image
    image_path = env.image()
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Crop the Petri dish
    cropped_image, bbox, success = crop_petri_dish(image, patch_size)
    if not success:
        raise RuntimeError("Failed to detect and crop the Petri dish.")

    # Split the image
    splits = split_image(cropped_image, num_parts=5)

    # Load the trained model
    model = load_model(model_path, custom_objects={"f1": f1, "dice_loss": dice_loss})

    # Initialize variables to store results
    root_tips_mm = []
    root_tips_robot = []

# Process each split image
for idx, split in enumerate(splits):
    print(f"Processing split {idx + 1} of {len(splits)}")

    try:
        # Generate the mask for the split
        mask = generate_mask(split, model, patch_size)

        # Detect the root tip using the updated skeletonization
        root_tip_pixel = detect_root_tip_with_skeletonization(
            mask, kernel_size=10, closing_iterations=3, min_area=400
        )

        # Convert root tip pixel coordinates to millimeters
        root_tip_mm = convert_pixel_to_mm(
            root_tip_pixel, image_height // len(splits), plate_height_mm // len(splits)
        )
        root_tips_mm.append(root_tip_mm)

        # Convert to robot coordinates
        root_tip_robot = convert_to_robot_coordinates(root_tip_mm, plate_position_robot)
        root_tips_robot.append(root_tip_robot)

        print(f"Split {idx + 1} root tip in robot coordinates: {root_tip_robot}")

    except Exception as e:
        print(f"Warning: No root detected in split {idx + 1}. Skipping to the next split. Error: {e}")
        continue  # Skip to the next split

    # Use the PID controller for inoculation
    print("Starting inoculation process...")
    inoculate_with_pid(env, root_tips_robot)
    print("Inoculation process completed.")


    # Close the environment after execution
    env.close()
   


Image captured and saved at: textures/_plates/035_43-17-ROOT1-2023-08-08_mock_pH5_+Fe_Col0_04-Fish Eye Corrected.png
Processing split 1 of 5
2/2 [==============================] - 3s 82ms/step
Split 1 root tip in robot coordinates: [0.12448179396092363, 0.13771936056838363, 0.057]
Starting inoculation process...
Starting inoculation for root tip 1 at [0.12448179396092363, 0.13771936056838363, 0.057]
Current position: [0.073  0.0895 0.1195], Goal position: [0.12448179 0.13771936 0.057     ]
Step 1 called.
Rendering: Goal Position [0.12448179 0.13771936 0.057     ]
XY Control: Current XY: [0.073  0.0895], Target XY: [0.12448179 0.13771936], Error XY: 0.07053709454148448
Current position: [0.0732 0.0897 0.1195], Goal position: [0.12448179 0.13771936 0.057     ]
Step 2 called.
Rendering: Goal Position [0.12448179 0.13771936 0.057     ]
XY Control: Current XY: [0.0732 0.0897], Target XY: [0.12448179 0.13771936], Error XY: 0.07025440404918439
Current position: [0.0735 0.0899 0.1195], Goal po